In [40]:
import math
import torch
import gpytorch
import numpy as np
from gpytorch.utils.lanczos import lanczos_tridiag
from gpytorch.utils.lanczos import lanczos_tridiag_to_diag

In [41]:
from gpytorch.kernels import RBFKernel

In [42]:
nx = 100
xx = torch.linspace(0, 5, nx)
kern = RBFKernel()
mat = kern(xx, xx).evaluate()

In [43]:
e_val, e_vec = np.linalg.eig(mat.detach())
idx = e_val.argsort()[::-1]   
e_val = torch.FloatTensor(e_val[idx].real)
e_vec = torch.FloatTensor(e_vec[:,idx].real)

### Lanczos Version

In [44]:
def multer(vec):
    return mat.matmul(vec)

In [62]:
n_vals = 20
Q, T = lanczos_tridiag(multer, n_vals, device="cpu", dtype=mat.dtype, 
                      matrix_shape=mat.shape)

In [63]:
t_eval, t_evec = lanczos_tridiag_to_diag(T)

In [64]:
t_eval = t_eval.detach().numpy()
t_evec = t_evec.detach().numpy()
idx = t_eval.argsort()[::-1]   
t_eval = torch.FloatTensor(t_eval[idx].real)
t_evec = torch.FloatTensor(t_evec[:,idx].real)

In [65]:
t_eval - e_val[:n_vals] ## these match pretty well, but not perfectly

tensor([ 3.8147e-06,  3.8147e-06,  0.0000e+00,  9.5367e-07,  1.9073e-06,
        -1.1921e-06,  2.3842e-07,  4.6896e-01,  3.4187e-01,  1.2762e-01,
         4.3151e-02,  1.3322e-02,  3.7810e-03,  9.9188e-04,  2.4173e-04,
         5.5101e-05,  1.1680e-05,  1.4517e-06,  2.8181e-07, -1.0800e-06])

In [66]:
t_evec = Q.matmul(t_evec)

In [77]:
evec_dot = t_evec.t().matmul(e_vec[:, :n_vals]) ## this looks mostly right